In [1]:
from pandas import set_option, DataFrame as DF, MultiIndex, Series, to_datetime

from main import (
    get_treasury_portfolio,
    get_treasury,
    get_historical_price_by_symbol,
    get_token_transfers_for_wallet,
    portfolio_filler
    )

In [2]:
portfolio = await get_treasury_portfolio("0x1a9C8182C09F50C8318d769245beA52c32BE35BC")

In [3]:
uni_v2_treasury_vested = await get_treasury(portfolio)

In [4]:
set_option("display.precision", 6)
treasury_assets = DF(data=uni_v2_treasury_vested.assets, index=[asset["token_symbol"] for asset in uni_v2_treasury_vested.assets])
treasury_assets.drop(["token_name", "token_symbol"], axis=1, inplace=True)
treasury_assets.rename_axis("token_symbol", inplace=True)

treasury_assets

,token_address,balance
token_symbol,,
UNI,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,1.113827e+09
ETH,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,3.338421e+00
DOTC,0xc0748cf90e231b8f21f75b7ad69732f19c76b751,1.545608e+00
ACE,0xec5483804e637d45cde22fa0869656b64b5ab1ab,1.401753e+00
SDOG,0x537edd52ebcb9f48ff2f8a28c51fcdb9d6a6e0d4,4.304749e-05
WJXN,0xca1262e77fb25c0a4112cfc9bad3ff54f617f2e6,1.017373e+01


In [5]:
treasury_assets.to_csv("uni_treasury_assets.csv")

In [6]:
asset_transfers = [
    await get_token_transfers_for_wallet("0x1a9C8182C09F50C8318d769245beA52c32BE35BC", token_address)
    for token_address in treasury_assets["token_address"]
]

In [13]:
from dateutil import parser
historical_treasury_balance: list[DF] = []

for trans_history in asset_transfers:
    if not trans_history:
        continue
    blocks = trans_history["items"]
    balances = []
    timeseries = []
    address = ""
    symbol = ""
    curr_balance = 0.0
    end_index = len(blocks) - 1
    for i in range(end_index, -1, -1):
        transfers = blocks[i]["transfers"]
        if i == end_index:
            address = transfers[0]["contract_address"]
            symbol = transfers[0]["contract_ticker_symbol"]

        for transfer in transfers:
            if not transfer["quote_rate"]:
                continue
            delta = int(transfer["delta"])
            if transfer["transfer_type"] == "IN":
                curr_balance += delta / 1e18
                balances.append(curr_balance)
            else:
                curr_balance -= delta / 1e18
                balances.append(curr_balance)

            timeseries.append(parser.parse(transfer["block_signed_at"]))

    index = MultiIndex.from_tuples(
        [
            (ts, address, symbol)
            for ts in timeseries
        ],
        names=["timestamp", "contract_address", "contract_symbol"] 
    )

    balances = Series(
        balances,
        index=index,
        name="treasury_balances"
    )

    if len(balances) > 0:
        historical_treasury_balance.append(balances)
    

/tmp/ipykernel_27165/1819984062.py:41: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  balances = Series(
/tmp/ipykernel_27165/1819984062.py:41: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  balances = Series(
/tmp/ipykernel_27165/1819984062.py:41: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  balances = Series(


In [8]:
historical_price_for_portfolio_assets = []
for symbol in treasury_assets.index.values:
    covalent_resp = await get_historical_price_by_symbol(
        symbol,
        (2, "years")
    )

    if not covalent_resp:
        continue

    indexes = MultiIndex.from_tuples(
        [
            (ts, address, symbol) for ts, address
            in zip(
                [ price["date"] for price in covalent_resp["prices"] ],
                [ price["contract_metadata"]["contract_address"] for price in covalent_resp["prices"] ]
            )
        ],
        names=["timestamp", "address", "symbol"]
    )

    historical_price_for_portfolio_assets.append(
        DF(
            data=covalent_resp["prices"],
            index=indexes
        )
        .drop(["contract_metadata", "date"], axis=1)
    )

historical_price_for_portfolio_assets[0]

,,,price
timestamp,address,symbol,
2022-05-13,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,4.719530
2022-05-12,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,4.621089
2022-05-11,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,5.229900
2022-05-10,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,5.939801
2022-05-09,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,6.834505
...,...,...,...
2020-09-20,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,5.332999
2020-09-19,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,5.623744
2020-09-18,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,7.106903


In [9]:
from math import log
cleaned_hist_prices = []
for df in historical_price_for_portfolio_assets:
    address_level = df.index.get_level_values(1)
    address = address_level[0]
    symbol_level = df.index.get_level_values(2)
    symbol = symbol_level[0]
    df = df.reset_index()

    """ `returns` calculation section

        `returns` = ln(current_price / previous_price)
    """
    returns = []
    for i in range(1, len(df)):
        try:
            # current price is df[i - 1] since `loc` descends the DF
            returns.append(log(df.loc[i - 1, "price"] / df.loc[i, "price"]))
        except Exception as e:
            print("misbehaving case: \n")
            print(f"\tsymbol: {symbol}\n\tindex: {i}\n\tcurr_price: {df.loc[i - 1, 'price']}\n\tprev_price: {df.loc[i, 'price']}")
            returns.append(None)
    returns.append(0)
    df["returns"] = returns

    """ end section
    """

    """ rolling std_dev of `returns` section

        period/window can be conifgurable, 7 days was set
    """

    window = 7
    rolling_window = df["returns"].iloc[::-1].rolling(window)
    std_dev = rolling_window.std(ddof=0)
    df["std_dev"] = std_dev

    """ end section
    """

    df = df.iloc[::-1]

    df.to_csv(
        f"hist_prices/{str(symbol) + '_' + str(address)[0:6] + '_hist_price'}.csv"
    )
    cleaned_hist_prices.append(df)

misbehaving case: 

	symbol: ACE
	index: 27
	curr_price: 0.0
	prev_price: 4.4383885e-08
misbehaving case: 

	symbol: SDOG
	index: 219
	curr_price: 0.0
	prev_price: 4.67809e-06
misbehaving case: 

	symbol: SDOG
	index: 223
	curr_price: 0.0
	prev_price: 5.6928975e-06


/tmp/ipykernel_27165/3814322006.py:18: RuntimeWarning: invalid value encountered in double_scalars
  returns.append(log(df.loc[i - 1, "price"] / df.loc[i, "price"]))
/tmp/ipykernel_27165/3814322006.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  returns.append(log(df.loc[i - 1, "price"] / df.loc[i, "price"]))
/tmp/ipykernel_27165/3814322006.py:18: RuntimeWarning: invalid value encountered in double_scalars
  returns.append(log(df.loc[i - 1, "price"] / df.loc[i, "price"]))


In [12]:
uni_prices = cleaned_hist_prices[0]

quote_rates = Series(data=uni_prices["price"])
quote_rates.index = to_datetime(uni_prices["timestamp"])

balances = portfolio_filler(historical_treasury_balance[0], quote_rates)

balances.to_csv("uni_tres_UNI_balance.csv")

price for 2022-04-19: 9.392122
balance:  227040881.18949774
